In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_teddynote import logging
logging.langsmith("CH21-AGENT")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH21-AGENT


# Binding Tools 

- LLM 모델이 도구를 호출 위해 chat 요청을 할 때, tool schema 전달 필요  

## 도구 정의 

해당 노트북에서 실험할 도구 정의 
- `get_word_length` : 단어의 길이를 반환하는 함수
- `add_function` : 두 숫자를 더하는 함수
- `naver_news_crawl` : 네이버 뉴스 기사를 크롤링하여 본문 내용을 반환하는 함수

In [3]:
import re 
import requests 
from bs4 import BeautifulSoup
from langchain.agents import tool

In [4]:
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word.
    """
    return len(word)

In [5]:
@tool
def add_function(a: float, b: float) -> float:
    """Adds two numbers together.
    """
    return a + b

In [6]:
@tool
def naver_news_crawl(news_url: str) -> str:
    """Crawls a Naver (naver.com) news article and returns the body content.
    """
    # HTTP GET 요청 보내기
    response = requests.get(news_url)

    # 요청이 성공했는지 확인
    if response.status_code == 200:
        # BeautifulSoup을 사용하여 HTML 파싱
        soup = BeautifulSoup(response.text, "html.parser")

        # 원하는 정보 추출
        title = soup.find("h2", id="title_area").get_text()
        content = soup.find("div", id="contents").get_text()
        cleaned_title = re.sub(r"\n{2,}", "\n", title)
        cleaned_content = re.sub(r"\n{2,}", "\n", content)
    else:
        print(f"HTTP 요청 실패. 응답 코드: {response.status_code}")

    return f"{cleaned_title}\n{cleaned_content}"

In [7]:
tools = [get_word_length, add_function, naver_news_crawl]

In [8]:
tools

[StructuredTool(name='get_word_length', description='Returns the length of a word.', args_schema=<class 'langchain_core.utils.pydantic.get_word_length'>, func=<function get_word_length at 0x1201374c0>),
 StructuredTool(name='add_function', description='Adds two numbers together.', args_schema=<class 'langchain_core.utils.pydantic.add_function'>, func=<function add_function at 0x1201372e0>),
 StructuredTool(name='naver_news_crawl', description='Crawls a Naver (naver.com) news article and returns the body content.', args_schema=<class 'langchain_core.utils.pydantic.naver_news_crawl'>, func=<function naver_news_crawl at 0x120137380>)]

## `.bind_tools()`

- `.bind_tools()` : 도구 바인딩 
- `tool_calls` : 결과 저장
    - `name` : 도구 이름 
    - `args` : 도구에 전달되는 인자 

In [9]:
from langchain_openai import ChatOpenAI

- OpenAI : 단순 프롬프트 기반 모델 사용 시 
- ChatOpenAI : 대화형 목적으로 모델 사용 시 

| 항목 | OpenAI | ChatOpenAI |
|-----|--------|------------|
| 사용 모델 | GPT-3 계열 (기존 Completion API) | GPT-3.5, GPT-4 등 (Chat API) |
| 대화 형식 | 일반 텍스트 입력 | 시스템/사용자/페르소나 기반의 대화 메시지 포맷 | 
| 주요 사용 | 단순 프롬프트 -> 응답 | 멀티턴, 에이전트 기반 구조 |
| 내부 동작 | `openai.Completion.create()` | `openai.ChatCompletion.create()`

In [10]:
# 모델 
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

# 도구 바인딩 
llm_with_tools = llm.bind_tools(tools)

In [11]:
# 도구 실행 
llm_with_tools.invoke("'SKALA' 단어는 몇 자 인가요?").tool_calls

[{'name': 'get_word_length',
  'args': {'word': 'SKALA'},
  'id': 'call_gpo48uLLvmNHoFttDRBVoaHM',
  'type': 'tool_call'}]

In [12]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

도구 결과에서...
- `type` : 도구 이름
- `args` : 도구에 전달되는 인자 

In [13]:
# 도구 바인딩 & 도구 파서 
chain = llm_with_tools | JsonOutputToolsParser(tools=tools)

# 실행 
tool_call_results = chain.invoke("'SKALA' 단어는 몇 자 인가요?")
print(tool_call_results)

[{'args': {'word': 'SKALA'}, 'type': 'get_word_length'}]


In [14]:
print(tool_call_results, end="\n\n==========\n\n")

# 첫 번째 도구 호출 결과
single_result = tool_call_results[0]

print(single_result["type"]) # 도구 이름
print(single_result["args"]) # 도구에 전달되는 인자

[{'args': {'word': 'SKALA'}, 'type': 'get_word_length'}]


get_word_length
{'word': 'SKALA'}


### def `execute_tool_calls` 

- 도구를 찾아서 `args`에 전달하여 도구 실행 
    - `type` : 도구 이름 
    - `args` : 도구에 전달되는 인자 

In [15]:
def execute_tool_calls(tool_call_results):
    """
    도구 호출 결과를 실행하는 함수

    :param tool_call_results: 도구 호출 결과 리스트
    :param tools: 사용 가능한 도구 리스트
    """
    # 도구 호출 결과 리스트 순회
    for tool_call_result in tool_call_results:
        
        # 도구의 이름과 인자 추출
        tool_name = tool_call_result["type"]  # 도구의 이름(함수명)
        tool_args = tool_call_result["args"]  # 도구에 전달되는 인자

        # 도구 이름과 일치하는 도구를 찾아 실행
        # next() 함수를 사용하여 일치하는 첫 번째 도구를 확인
        matching_tool = next((tool for tool in tools if tool.name == tool_name), None)

        if matching_tool:
            # 일치하는 도구를 찾았다면 실행
            result = matching_tool.invoke(tool_args)
            
            # 실행 결과 출력
            print(f"[실행도구] {tool_name} [Argument] {tool_args}\n[실행결과] {result}")
        
        else:
            # 일치하는 도구를 찾지 못했다면 에러 메시지
            print(f"에러: {tool_name}에 해당하는 도구를 찾을 수 없습니다.")

In [16]:
execute_tool_calls(tool_call_results)

[실행도구] get_word_length [Argument] {'word': 'SKALA'}
[실행결과] 5


### bind_tools + Parser + Execution

- 도구 실행, 결과 파싱 및 실행을 하나의 체인을 묶어서 진행 
- 흐름 정의 
    1. 모델에 도구 바인딩 : `llm_with_tools`
    2. 도구 호출 결과 파싱 : `JsonOutputToolsParser`
    3. 도구 호출 결과 실행 : `execute_tool_calls`

In [17]:
chain = llm_with_tools | JsonOutputToolsParser(tools=tools) | execute_tool_calls

In [18]:
chain.invoke("'SKALA' 단어는 몇 자 인가요?")

[실행도구] get_word_length [Argument] {'word': 'SKALA'}
[실행결과] 5


In [19]:
chain.invoke("114.52 + 382.93")

[실행도구] add_function [Argument] {'a': 114.52, 'b': 382.93}
[실행결과] 497.45


In [20]:
chain.invoke("뉴스 기사 내용을 크롤링 해주세요: https://n.news.naver.com/mnews/article/001/0015275227")

[실행도구] naver_news_crawl [Argument] {'news_url': 'https://n.news.naver.com/mnews/article/001/0015275227'}
[실행결과] 앤스로픽 "한국에 지사 만들 것…올해 아태지역 투자 본격화"

국내 첫 행사 개최 
마이크 크리거 앤스로픽 최고제품책임자(CPO)(서울=연합뉴스) 조현영 기자 = 마이크 크리거 최고제품책임자(CPO)가 19일 서울 송파구 시그니엘에서 열린 '코리아 빌더 서밋'에서 발표하고 있다. 2025.3.19 hyun0@yna.co.kr    (서울=연합뉴스) 조현영 기자 = 케이트 젠슨 앤스로픽 매출 총괄 책임자는 19일 "인공지능(AI) 모델 '클로드'를 통해 기업이 문제를 해결하는 것을 지원하고자 한국에 왔다"며 "올해는 아시아태평양 지역에 본격적으로 투자하는 첫해가 될 것이며 한국에 지사도 만들 예정"이라고 말했다.    젠슨 책임자는 이날 거대 언어 모델(LLM) 모니터링 및 분석 플랫폼을 제공하는 한국 스타트업 콕스웨이브와 앤스로픽이 서울 송파구 시그니엘에서 공동 주최한 '코리아 빌더 서밋' 오프닝 세션에서 이같이 밝혔다.         앤스로픽은 챗GPT 개발사인 오픈AI 창립 멤버가 2021년 설립한 미국의 AI 스타트업으로, 클로드를 통해 아마존 등으로부터 대규모 자금을 유치하며 이름을 알렸다.     콕스웨이브는 클로드의 기능 등 앤스로픽 연구 성과를 한국에 소개하고, 앤스로픽과 한국 기업 간 접점을 늘리기 위해 서밋을 개최했다고 설명했다. 앤스로픽으로서는 한국에서 진행하는 첫 행사다.     이날 행사에서 마이크 크리거 최고제품책임자(CPO)는 최신 AI 모델 '클로드 3.7 소넷'을 소개했다. 크리거 CPO는 인스타그램의 공동 창업자다.     이 모델은 하이브리드 추론 모델로서, 뛰어난 코딩 능력과 인간과 협업에 특화한 강점을 바탕으로 국내 기업이 AI를 업무에 적용하는 데 도움을 줄 수 있다고 크리거 CPO는 설명했다.     그는 "클로드가 한국어를

## AgentExecutor 대체

- `bind_tools` : 모델이 사용할 수 있는 도구 제공 
- `AgentExecutor` : 실제로 LLM 호출, 올바른 도구로 라우팅/실행/모델 호출 등을 위한 실행 루프 생성 

In [21]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

In [22]:
# Agent 프롬프트 생성
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        (
            "user", 
            "{input}"
         ),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# 모델 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 도구 정의 
tools = [get_word_length, add_function, naver_news_crawl]

# Agent 정의 
agent = create_tool_calling_agent(llm, tools, prompt)

# AgentExecutor 생성 
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_erros=True,
)

In [23]:
# Agent 실행 
result = agent_executor.invoke({
    "input":"'SKALA' 단어는 몇 자 인가요?",
})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'SKALA'}`


5'SKALA' 단어는 5자입니다.

> Finished chain.


In [24]:
# 결과 확인 
print(result["output"])

'SKALA' 단어는 5자입니다.


In [25]:
result = agent_executor.invoke({
    "input":"298873.275 + 1876236.62 계산 결과는?",
})

print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `add_function` with `{'a': 298873.275, 'b': 1876236.62}`


2175109.895298873.275 + 1876236.62의 계산 결과는 2175109.895입니다.

> Finished chain.
298873.275 + 1876236.62의 계산 결과는 2175109.895입니다.


In [26]:
result = agent_executor.invoke({
    "input":"114.2 + 213.2 + 0.05 + 100.0 계산 결과는?",
})

print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `add_function` with `{'a': 114.2, 'b': 213.2}`


327.4
Invoking: `add_function` with `{'a': 0.05, 'b': 100.0}`


100.05
Invoking: `add_function` with `{'a': 327.4, 'b': 100.05}`


427.45계산 결과는 427.45입니다.

> Finished chain.
계산 결과는 427.45입니다.


In [27]:
result = agent_executor.invoke({
    "input":"뉴스 기사 내용을 100자 이내로 요약해 해주세요: https://n.news.naver.com/mnews/article/001/0015275227",
})

print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `naver_news_crawl` with `{'news_url': 'https://n.news.naver.com/mnews/article/001/0015275227'}`


앤스로픽 "한국에 지사 만들 것…올해 아태지역 투자 본격화"

국내 첫 행사 개최 
마이크 크리거 앤스로픽 최고제품책임자(CPO)(서울=연합뉴스) 조현영 기자 = 마이크 크리거 최고제품책임자(CPO)가 19일 서울 송파구 시그니엘에서 열린 '코리아 빌더 서밋'에서 발표하고 있다. 2025.3.19 hyun0@yna.co.kr    (서울=연합뉴스) 조현영 기자 = 케이트 젠슨 앤스로픽 매출 총괄 책임자는 19일 "인공지능(AI) 모델 '클로드'를 통해 기업이 문제를 해결하는 것을 지원하고자 한국에 왔다"며 "올해는 아시아태평양 지역에 본격적으로 투자하는 첫해가 될 것이며 한국에 지사도 만들 예정"이라고 말했다.    젠슨 책임자는 이날 거대 언어 모델(LLM) 모니터링 및 분석 플랫폼을 제공하는 한국 스타트업 콕스웨이브와 앤스로픽이 서울 송파구 시그니엘에서 공동 주최한 '코리아 빌더 서밋' 오프닝 세션에서 이같이 밝혔다.         앤스로픽은 챗GPT 개발사인 오픈AI 창립 멤버가 2021년 설립한 미국의 AI 스타트업으로, 클로드를 통해 아마존 등으로부터 대규모 자금을 유치하며 이름을 알렸다.     콕스웨이브는 클로드의 기능 등 앤스로픽 연구 성과를 한국에 소개하고, 앤스로픽과 한국 기업 간 접점을 늘리기 위해 서밋을 개최했다고 설명했다. 앤스로픽으로서는 한국에서 진행하는 첫 행사다.     이날 행사에서 마이크 크리거 최고제품책임자(CPO)는 최신 AI 모델 '클로드 3.7 소넷'을 소개했다. 크리거 CPO는 인스타그램의 공동 창업자다.     이 모델은 하이브리드 추론 모델로서, 뛰어난 코딩 능력과 인간과 협업에 특화한 강점을 바탕으로 국내 기업이 AI를 업무에 적용하는 데 도움을 줄 수

--------
** End of Documents **